In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

import requests

# URL of the file
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Download the file
r = requests.get(url)
with open('home_sales_revised.csv', 'wb') as f:
    f.write(r.content)

# Now read the downloaded file into a Spark DataFrame
home_sales_df = spark.read.csv('home_sales_revised.csv', header=True, inferSchema=True)

In [18]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")
home_sales_df.columns

['id',
 'date',
 'date_built',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view']

In [19]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query = """
SELECT 
    YEAR(date) AS year, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
"""

# Execute the query
avg_price_df = spark.sql(query)

# Show the results
avg_price_df.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [20]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# SQL Query
query = """
SELECT 
    date_built, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query
avg_price_per_year_built_df = spark.sql(query)

# Show the results
avg_price_per_year_built_df.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [21]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
SELECT 
    date_built, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query
avg_price_criteria_df = spark.sql(query)

# Show the results
avg_price_criteria_df.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [23]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

from pyspark.sql import functions as F

start_time = time.time()

query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""

# Execute the query
avg_price_view_df = spark.sql(query)

avg_price_view_df.show()
print("--- runtime = %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- runtime = 0.17539095878601074 seconds ---


In [37]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# First, run query as usual for uncached runtime
query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""

# Measure runtime for uncached data
start_time_uncached = time.time()
spark.sql(query).show()
end_time_uncached = time.time()
runtime_uncached = end_time_uncached - start_time_uncached


# Now, cache the temporary table
spark.catalog.cacheTable("home_sales")

# Measure runtime for cached data
start_time_cached = time.time()
result = spark.sql(query).collect() #using 'collect' to execute the query without showing
end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached

# Measure the difference
runtime_diff = runtime_uncached - runtime_cached


print(f"Runtime uncached: {runtime_uncached} seconds")
print(f"Runtime cached: {runtime_cached} seconds")
print (f"A difference of {runtime_diff} seconds")


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows



23/11/10 15:33:11 WARN CacheManager: Asked to cache already cached data.


TypeError: collect() takes 1 positional argument but 2 were given

In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.parquet("parquet_home_sales")

In [35]:
# 11. Read the formatted parquet data.
home_sales_parquet_df = spark.read.parquet("parquet_home_sales")

In [36]:
# 12. Create a temporary table for the parquet data.
home_sales_parquet_df.createOrReplaceTempView("home_sales_parquet_view")

In [38]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet_view 
WHERE price >= 350000
GROUP BY view
"""

# Measure runtime for the query on Parquet data
start_time_parquet = time.time()
spark.sql(query).show() 
end_time_parquet = time.time()
runtime_parquet = end_time_parquet - start_time_parquet

# Measure runtime for the query on Cached data 
# (replacing "home_sales_parquet_view" with "home_sales" to use the same query)
start_time_cached = time.time()
result = spark.sql(query.replace("home_sales_parquet_view", "home_sales")).collect() #using 'collect' to execute the query without showing
end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached

# Print the results
print(f"Runtime on Parquet data: {runtime_parquet} seconds")
print(f"Runtime on Cached data: {runtime_cached} seconds")
print(f"Difference in runtime: {abs(runtime_parquet - runtime_cached)} seconds")


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

Runtime on Parquet data: 0.2845158576965332 seconds
Runtime on Cached data: 0.07106924057006836 seconds
Difference in runtime: 0.21344661712646484 seconds


In [39]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [42]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False